In [1]:
import tensorflow as tf

In [2]:
from tensorflow import keras

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator #
from tensorflow.keras.models import Sequential #
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D 
#Dense=computational unit make less
#Dropout = Numerical unit drop

In [4]:
#cd /path
#!unzip /path or archive.zip
num_classes = 2 #cat and dog
img_row = 64
img_col = 64 #image size
train_dataset = "E:\\AIBroadway\\catvsdog\\training_set\\training_set"
test_dataset = "E:\\AIBroadway\\catvsdog\\test_set\\test_set"

In [5]:
#data augmentation
train_datagen = ImageDataGenerator(
rescale = 1./255,
rotation_range = 30,
width_shift_range = 0.4,
height_shift_range = 0.4,
horizontal_flip = True,
fill_mode='nearest')

In [6]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_dataset,
    target_size=(img_row,img_col),
    batch_size = 32,
    class_mode = 'binary',
    shuffle = True
)

Found 8005 images belonging to 2 classes.


In [8]:
validation_generator = train_datagen.flow_from_directory(
    test_dataset,
    target_size=(img_row,img_col),
    batch_size = 32,
    class_mode = 'binary',
    shuffle = True
)

Found 2023 images belonging to 2 classes.


In [9]:
model = Sequential() #feature map
#input layer for convolution
model.add(Conv2D(64,(3,3),padding='same',input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#hidden layer conv
#model.add(Conv2D(32,(3,3),padding='same'))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size = (2,2)))
#model.add(Dropout(0.25))

model.add(Conv2D(16,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#flatten conv
model.add(Flatten())#single layer 1D
model.add(Dense(256)) #only few send
model.add(Activation('relu'))
model.add(Dropout(0.5)) #size decrease
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 64)        1792      
_________________________________________________________________
activation (Activation)      (None, 64, 64, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 16)        9232      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 16)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 16)        0

In [10]:
#neural network
model.add(Dense(units=256, activation = 'relu'))
model.add(Dense(units=128, activation = 'relu'))
model.add(Dense(units=32, activation = 'relu'))
model.add(Dense(units=16, activation = 'relu')) # only +
model.add(Dense(1))
model.add(Activation('sigmoid')) #0 or 1 output layer ma yes or no softmax do for probality
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 64)        1792      
_________________________________________________________________
activation (Activation)      (None, 64, 64, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 16)        9232      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 16)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 16)        0

In [11]:
#comile before traing
#training
model.compile(loss='binary_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

model.fit_generator(
    train_generator,
    steps_per_epoch = 8005//32,
    epochs = 5,
    validation_data= validation_generator,
    validation_steps = 2023//32
)

D:\Other\anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
250/250 [==============================] - 298s 1s/step - loss: 0.6958 - accuracy: 0.5067 - val_loss: 0.6933 - val_accuracy: 0.4995
Epoch 2/5
250/250 [==============================] - 57s 228ms/step - loss: 0.6952 - accuracy: 0.4917 - val_loss: 0.6920 - val_accuracy: 0.5025
Epoch 3/5
250/250 [==============================] - 57s 226ms/step - loss: 0.6899 - accuracy: 0.5497 - val_loss: 0.6794 - val_accuracy: 0.5828
Epoch 4/5
250/250 [==============================] - 56s 224ms/step - loss: 0.6826 - accuracy: 0.5659 - val_loss: 0.6719 - val_accuracy: 0.5769
Epoch 5/5
250/250 [==============================] - 58s 230ms/step - loss: 0.6734 - accuracy: 0.5833 - val_loss: 0.6724 - val_accuracy: 0.5729


In [11]:
model.save('cat_vs_dog.h5')

In [13]:
#test
from tensorflow.python.keras.utils.generic_utils import class_and_config_for_serialized_keras_object
import numpy as np #to convert array
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

classifier = load_model('E:\\AIBroadway\\cat_vs_dog.h5')
test_image = image.load_img('E:\\AIBroadway\\catvsdog\\test_set\\test_set\\cats\\cat.4002.jpg',target_size = (64,64,3))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)
result = classifier.predict(test_image)
print(result)
if result > 0.6:
  print("This is dog")
else:
  print("This is cat")

[[0.9999722]]
This is dog
